# 実行環境の作成

## Google Cloud Storage

下記コードでGCPに接続

In [9]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [11]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/14 00:14:37.895541 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/14 00:14:37.913177 Opening GCS connection...
2022/09/14 00:14:38.270803 Mounting file system "statistics-hyogo"...
2022/09/14 00:14:38.271397 File system has been successfully mounted.


## Google Secret Maneger

ライブラリをインストール

In [ ]:
pip install google-cloud-secret-manager

シークレットを呼び出す関数

In [6]:
from google.cloud import secretmanager

def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

ESTAT_APPIDを取得する

In [8]:
PROJECT_ID = 'primal-buttress-342908'
ESTAT_APPID = access_secret(PROJECT_ID,'ESTAT_APPID' )
# print(ESTAT_APPID)

## pythonのライブラリ

In [9]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

統計表を指定してメタ情報を取得する

[API仕様](https://www.e-stat.go.jp/api/api-info/e-stat-manual3-0#api_2_2)

'statsDataId'に統計表コードを指定

In [3]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [10]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
# print(meta)

## カテゴリコード一覧

カテゴリコード'cat01'のリストを作成する関数

In [ ]:
def get_estat_catefory_codes(statsDataId):
  meta= get_estat_meta(statsDataId)
 
  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # カテゴリ一覧
  categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # カテゴリコード一覧
  catagory_codes = [d.get('@code') for d in categories]

  return catagory_codes

category_codes = get_estat_catefory_codes(statsDataId)
# print(category_codes)

## 地域コード一覧

地域コード'area'のリスト一覧を作成する関数

全国'00000'は除外している

In [ ]:
def get_estat_area_codes(statsDataId):
  # estat-APIのメタ情報を取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # 地域一覧
  areas = next((d for d in CLASS_OBJ if d['@id'] == 'area'), None)['CLASS']

  # 地域コード一覧（全国'00000'は削除）
  area_codes = [d.get('@code') for d in areas]
  area_codes.remove('00000')

  return area_codes

area_codes = get_estat_area_codes(statsDataId)
print(area_codes)

# estat-APIのデータを取得

## 総人口のデータ取得

まずは、総人口のデータを取得してみる

cat01に'A1101'を指定
areaは全都道府県を指定

リストを指定する場合はカンマ区切りの文字列に変換する

In [22]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1101'
params['cdArea']=",".join(area_codes)

# print (params)

estat-APIのデータを取得する関数を作成

In [ ]:
def get_estat_data(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

res = get_estat_data(params)
# print(res)

## pandas DataFrameに変換

estat-APIのレスポンスからVALUEを抜き出し、DataFrameに変換する

In [ ]:
res = get_estat_data(params)

# VALUE
VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df_value = pd.json_normalize(VALUE)
# print(df_value)

カテゴリ情報は'CLASS_OBJ'に格納されている。

これもDataFrameに変換

In [32]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

df_categories = pd.json_normalize(categories)
# print(df_categories)

   @code      @name @level @unit
0  A1101  A1101_総人口      1     人


'VALUE'と'CLASS_OBJ'をDataFrameで外部結合する関数を作成

In [ ]:
def get_estat_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # VALUE
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  df_value = pd.json_normalize(VALUE)

  for d in CLASS_OBJ :
    # キー名を取得 @time,@area,,,
    key_name = '@{}'.format(d['@id'])

    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_value = pd.merge(df_value, df_class, on=key_name, how='outer')
  
  return df_value

df_org = get_estat_dataframe(res)
print(df_org)

## DataFrameの整形

使い勝手が良くなるように、DataFrameを整形していく

必要な列だけ抽出する

In [ ]:
df_1 = df_org[['@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

# print(df_1)

列名を変更する

In [ ]:
columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
df_1 = df_1.rename(columns=columns)

# print(df_1)

欠損値が含まれる行を削除する

In [54]:
df_1 = df_1.dropna()

# print(df_1)

categoryNameから不要な情報（categoryCode）を削除する

In [ ]:
df_1['categoryName'] = df_1.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

# print(df_1)

timeCodeを4桁文字列に置換する

In [ ]:
df_1['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

print(df_1)

ここまでを関数にまとめる

In [ ]:
def  format_estat_dataframe(df_org):
  # 必要な列だけ抽出
  df =  df_org[['@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
  df = df.rename(columns=columns)

  # 欠損データ削除
  df = df.dropna()

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

  return df

df_fmt = format_estat_dataframe(df_org)
df_fmt

## 順位を付与する

年度ごとに順位を付与

In [104]:
def get_estat_dataframe_withrank(df_arg):
  df = df_arg.copy()

  # 年度リストを作成
  times = df['timeCode'].tolist()
  times = list(set(times))

  # 年度毎に順位を付与
  for time in times:
    df_1 = df.copy()[df['timeCode'] == time]
    df_1['value'].astype(int)
    print(df_1)
    df['rank'] = df_1.rank(ascending=False)['value'].astype(int).astype(str)


  return df



df_last = get_estat_dataframe_withrank(df_fmt)
# print(df_last)

     categoryCode categoryName timeCode timeName areaCode areaName     value  \
1692        A1101          総人口     2011   2011年度    01000      北海道   5488000   
1693        A1101          総人口     2011   2011年度    02000      青森県   1363000   
1694        A1101          総人口     2011   2011年度    03000      岩手県   1315000   
1695        A1101          総人口     2011   2011年度    04000      宮城県   2326000   
1696        A1101          総人口     2011   2011年度    05000      秋田県   1075000   
1697        A1101          総人口     2011   2011年度    06000      山形県   1162000   
1698        A1101          総人口     2011   2011年度    07000      福島県   1988000   
1699        A1101          総人口     2011   2011年度    08000      茨城県   2960000   
1700        A1101          総人口     2011   2011年度    09000      栃木県   2000000   
1701        A1101          総人口     2011   2011年度    10000      群馬県   2001000   
1702        A1101          総人口     2011   2011年度    11000      埼玉県   7209000   
1703        A1101          総人口     2011 

In [66]:

# 年次一覧リストを作成
times = df_value['timeCode'].tolist()
times = list(set(times))
print(len(times))

# 最終DataFrame
cols = ['categoryCode', 'categoryName','timeCode','timeName','areaCode','areaName','value','unit','rank']
df_last = pd.DataFrame(index=[], columns=cols)
print(df_last)

# 順位を付与して、最終DataFrameに整形する
for time in times:
  df = df_value.copy()[df_value['timeCode'] == time]
  df['rank'] = df.rank(ascending=False)['value'].astype(int).astype(str)
  df['value'] = df['value'].astype(str)
  df_last = pd.concat([df_last, df])

print(df_last)

KeyError: ignored